In [1]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('all')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downlo

True

In [2]:
# Install required packages
!pip install -q langchain langchain_community pypdf matplotlib pandas plotly seaborn reportlab nltk faiss-cpu sentence-transformers duckduckgo-search transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.

In [3]:
# RFP Risk Analysis Agent
# Complete implementation for Google Colab

# Install all required dependencies
import os
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from IPython.display import HTML, display
from nltk.tokenize import sent_tokenize
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from datetime import datetime
from google.colab import files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from transformers import pipeline
from duckduckgo_search import DDGS

# Download NLTK data
nltk.download('punkt', quiet=True)

# Initialize the text classification model
risk_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Contract clause categories based on provided information
contract_categories = {
    "Parties": ["party", "parties", "between", "agreement between"],
    "Agreement Date": ["agreement date", "dated", "as of", "effective as of"],
    "Effective Date": ["effective date", "comes into effect", "shall be effective"],
    "Expiration Date": ["expiration date", "term", "expires", "termination date"],
    "Renewal Term": ["renewal", "extend", "extension", "renewed", "automatic renewal"],
    "Notice to Terminate": ["notice", "terminate", "termination notice", "days notice", "written notice"],
    "Governing Law": ["governing law", "jurisdiction", "governed by the laws", "applicable law"],
    "Most Favored Nation": ["most favored", "favored nation", "best terms", "equal terms", "better terms"],
    "Non-Compete": ["non-compete", "not compete", "competition", "competing", "competitive"],
    "Exclusivity": ["exclusive", "exclusively", "sole", "exclusively dealing", "requirements"],
    "No-Solicit of Customers": ["solicit", "solicitation", "customers", "clients", "not solicit"],
    "Competitive Restriction": ["exception", "carve-out", "carveout", "notwithstanding", "excluding"],
    "No-Solicit of Employees": ["employees", "personnel", "staff", "recruit", "hiring"],
    "Non-Disparagement": ["disparage", "disparagement", "negative statement", "reputation"],
    "Termination for Convenience": ["convenience", "without cause", "at any time", "sole discretion"],
    "ROFR/ROFO/ROFN": ["right of first", "ROFR", "ROFO", "ROFN", "first refusal", "first offer"],
    "Change of Control": ["change of control", "merger", "acquisition", "sale of assets"],
    "Anti-Assignment": ["assignment", "assign", "not assign", "consent to assign"],
    "Revenue/Profit Sharing": ["revenue sharing", "profit sharing", "royalty", "commission"],
    "Price Restriction": ["price", "pricing", "rate", "increase price", "price changes"],
    "Minimum Commitment": ["minimum", "at least", "no less than", "minimum order", "minimum purchase"],
    "Volume Restriction": ["volume", "quantity", "threshold", "exceeds", "limitation"],
    "IP Ownership": ["intellectual property", "IP", "ownership", "assign", "transfer"],
    "Joint IP Ownership": ["joint ownership", "jointly own", "share ownership", "co-own"],
    "License Grant": ["license", "licensed", "grants", "grant", "right to use"],
    "Non-Transferable License": ["non-transferable", "not transfer", "no sublicense"],
    "Affiliate License": ["affiliate", "subsidiary", "parent company", "related entity"],
    "Unlimited License": ["unlimited", "enterprise-wide", "all you can eat", "unrestricted"],
    "Irrevocable License": ["irrevocable", "perpetual", "cannot be revoked", "permanent"],
    "Source Code Escrow": ["escrow", "source code", "deposit", "third-party escrow"],
    "Post-Termination": ["post-termination", "after termination", "survival", "wind-down"],
    "Audit Rights": ["audit", "inspect", "examination", "records", "books"],
    "Uncapped Liability": ["uncapped", "unlimited liability", "no limitation of liability"],
    "Cap on Liability": ["cap", "limitation of liability", "limited to", "maximum liability"],
    "Liquidated Damages": ["liquidated damages", "penalty", "predetermined damages", "termination fee"],
    "Warranty Duration": ["warranty", "warranties", "guarantee", "defects", "errors"],
    "Insurance": ["insurance", "insure", "coverage", "policy", "insured"],
    "Covenant Not to Sue": ["covenant not to sue", "no claim", "waive right", "will not bring suit"],
    "Third Party Beneficiary": ["third party", "beneficiary", "third-party beneficiary", "benefit of"]
}

# Risk categories for risk analysis
risk_categories = {
    "Termination": ["terminate", "termination", "cancel", "cancellation", "end contract", "fire", "dismiss", "discontinue"],
    "Liability": ["liability", "indemnification", "indemnify", "harmless", "damages", "claims", "legal action", "negligence", "sue"],
    "Financial": ["payment", "penalty", "fee", "pricing", "cost", "expense", "budget", "financial", "monetary", "compensation"],
    "Timeline": ["deadline", "timeline", "schedule", "delay", "milestone", "delivery date", "timeframe", "duration"],
    "Compliance": ["comply", "compliance", "regulation", "regulatory", "law", "legal", "requirement", "standard", "certification"],
    "Scope Creep": ["scope", "additional", "extra", "change request", "modification", "alter", "amendment", "expand"],
    "Intellectual Property": ["IP", "intellectual property", "copyright", "trademark", "patent", "proprietary", "ownership", "license"],
    "Data Security": ["data", "security", "privacy", "confidential", "breach", "protection", "GDPR", "encryption", "sensitive"],
}

# Risk severity levels
risk_severity = {
    "High": {
        "color": colors.red,
        "score": 3,
        "description": "Significant risk that requires immediate attention",
        "time_sensitivity": "Immediate action required before proposal submission"
    },
    "Medium": {
        "color": colors.orange,
        "score": 2,
        "description": "Moderate risk that should be addressed in the proposal response",
        "time_sensitivity": "Address before contract signing"
    },
    "Low": {
        "color": colors.green,
        "score": 1,
        "description": "Monitor during project execution; no immediate action required",
        "time_sensitivity": "Can be monitored during project execution"
    }
}

# Time sensitivity assessment
time_sensitivity = {
    "Immediate": {
        "color": colors.red,
        "description": "Must be addressed before proceeding with proposal",
        "deadline": "Before proposal submission"
    },
    "Short-term": {
        "color": colors.orange,
        "description": "Should be addressed before contract signing",
        "deadline": "Before contract signing"
    },
    "Medium-term": {
        "color": colors.yellowgreen,
        "description": "Can be addressed during project setup",
        "deadline": "Before project start"
    },
    "Long-term": {
        "color": colors.green,
        "description": "Can be monitored during project execution",
        "deadline": "During project execution"
    }
}

# Function to perform web search
def perform_web_search(query, max_results=3):
    """Perform a web search to get context about a risk or clause."""
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=max_results))

        if not results:
            return "No search results found."

        formatted_results = []
        for i, result in enumerate(results, 1):
            title = result.get('title', 'No Title')
            body = result.get('body', 'No content available')
            formatted_results.append(f"{i}. {title}: {body}")

        return "\n\n".join(formatted_results)
    except Exception as e:
        print(f"Search error: {str(e)}")
        return "Search functionality unavailable. Using built-in knowledge."

# Function to get legal context about a specific clause type
def get_legal_context(clause_text, category):
    """Get legal context about a specific type of clause."""
    search_queries = {
        "Termination": f"legal implications of {category.lower()} clause in contracts",
        "Liability": f"standard {category.lower()} clauses in contracts legal analysis",
        "Financial": f"fair {category.lower()} terms in business contracts",
        "Timeline": f"reasonable {category.lower()} in business contracts",
        "Compliance": f"{category.lower()} requirements in contracts legal standards",
        "Scope Creep": f"protecting against {category.lower()} in contracts",
        "Intellectual Property": f"standard {category.lower()} clauses in contracts",
        "Data Security": f"{category.lower()} best practices in contracts"
    }

    # Extract key terms from the clause
    key_terms = extract_key_terms(clause_text)

    # Create a targeted search query
    if category in search_queries:
        query = f"{search_queries[category]} {key_terms}"
    else:
        query = f"contract clause risks {key_terms}"

    # Perform the search
    return perform_web_search(query)

# Function to upload an RFP document
def upload_rfp_document():
    print("Please upload the RFP document (PDF format)...")
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded.")
        return None

    # Get the filename of the uploaded file
    filename = list(uploaded.keys())[0]
    return filename

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        # Use PyPDFLoader from LangChain
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()

        # Extract text from each page
        text = ""
        for page in pages:
            text += page.page_content + "\n"

        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {str(e)}")
        return ""

# Extract key terms from a clause or text
def extract_key_terms(text):
    """Extract important keywords from text for search."""
    # Common stop words to filter out
    stop_words = [
        "the", "a", "an", "and", "or", "but", "if", "in", "on", "at",
        "to", "for", "with", "by", "about", "as", "into", "like", "of",
        "from", "shall", "will", "must", "should", "may", "can", "could",
        "would", "is", "are", "was", "were", "be", "been", "being",
        "have", "has", "had", "do", "does", "did", "not"
    ]

    # Extract words and filter
    words = re.findall(r'\b[a-zA-Z]{3,}\b', text.lower())
    filtered_words = [word for word in words if word not in stop_words]

    # Get the most frequent meaningful words
    from collections import Counter
    word_counts = Counter(filtered_words)
    common_words = [word for word, count in word_counts.most_common(5)]

    return " ".join(common_words)

# Function to extract contract clauses
def extract_clauses(text):
    # Split text into sentences
    sentences = sent_tokenize(text)

    # Identify potential clause sentences
    clauses = []

    for sentence in sentences:
        # Look for sentences that might be clauses
        if any(keyword in sentence.lower() for category in risk_categories.values() for keyword in category) or \
           any(keyword in sentence.lower() for category in contract_categories.values() for keyword in category):
            clauses.append(sentence)

    return clauses

# Identify contract clause types
def identify_clause_types(clause):
    """Identify the type of contract clause."""
    clause_types = []

    for category, keywords in contract_categories.items():
        if any(keyword in clause.lower() for keyword in keywords):
            clause_types.append(category)

    return clause_types

# Enhanced function to identify high-risk terms
def is_high_risk_term(clause_text):
    high_risk_terms = [
        "without notice", "sole discretion", "no liability",
        "unlimited liability", "immediate termination",
        "waive right", "non-negotiable", "full responsibility",
        "no compensation", "forfeit", "penalty", "irrevocable",
        "absolute right", "without cause", "unilateral",
        "hold harmless", "unreasonable", "excessive"
    ]

    # Check for high risk terms
    for term in high_risk_terms:
        if term in clause_text.lower():
            return True, term

    return False, None

# Identify potential time-sensitive clauses
def is_time_sensitive(clause_text):
    # Time-sensitive indicators
    immediate_action_indicators = [
        "immediate", "urgent", "promptly", "without delay",
        "emergency", "critical", "within 24 hours", "expedite"
    ]

    short_term_indicators = [
        "within 7 days", "one week", "prior to submission",
        "before proposal", "initial phase"
    ]

    medium_term_indicators = [
        "within 30 days", "month", "project setup",
        "implementation phase", "initial delivery"
    ]

    long_term_indicators = [
        "ongoing", "throughout the project", "maintenance",
        "over time", "periodic", "regular review"
    ]

    # Check for time sensitivity
    if any(indicator in clause_text.lower() for indicator in immediate_action_indicators):
        return "Immediate"
    elif any(indicator in clause_text.lower() for indicator in short_term_indicators):
        return "Short-term"
    elif any(indicator in clause_text.lower() for indicator in medium_term_indicators):
        return "Medium-term"
    else:
        return "Long-term"

# # Function to identify risky clauses
# def identify_risky_clauses(clauses):
#     risky_clauses = []

#     print("Analyzing clauses for potential risks...")

#     for i, clause in enumerate(clauses):
#         if i % 20 == 0:
#             print(f"Processed {i} of {len(clauses)} clauses...")

#         # Determine which risk categories apply
#         risk_cats = []
#         for category, keywords in risk_categories.items():
#             if any(keyword in clause.lower() for keyword in keywords):
#                 risk_cats.append(category)

#         # Determine which contract categories apply
#         contract_cats = identify_clause_types(clause)

#         # Only proceed if we have a risk or contract category
#         if risk_cats or contract_cats:
#             # Use sentiment analysis as a risk detector
#             sentiment = risk_classifier(clause)[0]

#             # Check for high-risk terms
#             is_high_risk, high_risk_term = is_high_risk_term(clause)

#             # Assess time sensitivity
#             time_frame = is_time_sensitive(clause)

#             # Determine risk level with updated logic
#             if is_high_risk or (sentiment["label"] == "NEGATIVE" and sentiment["score"] > 0.75):
#                 risk_level = "High"
#             elif sentiment["score"] > 0.6 or any(cat in ["Liability", "Termination", "Intellectual Property"] for cat in risk_cats):
#                 risk_level = "Medium"
#             else:
#                 # Lower risk for clauses that can be monitored or addressed later
#                 risk_level = "Low" if time_frame in ["Medium-term", "Long-term"] else "Medium"

#             # Adjust risk level based on contract categories
#             high_risk_contract_categories = [
#                 "Uncapped Liability", "Termination for Convenience", "Non-Compete",
#                 "Exclusivity", "IP Ownership", "Minimum Commitment"
#             ]

#             if any(cat in high_risk_contract_categories for cat in contract_cats) and risk_level != "High":
#                 risk_level = "Medium"  # At minimum, these are medium risk

#             # Get web search context for the primary category (if available)
#             web_context = ""
#             primary_category = risk_cats[0] if risk_cats else (contract_cats[0] if contract_cats else "General")

#             # Only do web search for higher risk items to avoid rate limiting
#             if risk_level in ["High", "Medium"]:
#                 try:
#                     web_context = get_legal_context(clause, primary_category)
#                 except Exception as e:
#                     print(f"Error during web search: {str(e)}")
#                     web_context = "Web search unavailable."

#             risky_clauses.append({
#                 "clause": clause,
#                 "risk_categories": risk_cats,
#                 "contract_categories": contract_cats,
#                 "risk_level": risk_level,
#                 "sentiment_score": sentiment["score"],
#                 "high_risk_term": high_risk_term if is_high_risk else None,
#                 "time_sensitivity": time_frame,
#                 "web_context": web_context
#             })

#     return risky_clauses


# Function to identify risky clauses
def identify_risky_clauses(clauses):
    risky_clauses = []

    print("Analyzing clauses for potential risks...")

    for i, clause in enumerate(clauses):
        if i % 20 == 0:
            print(f"Processed {i} of {len(clauses)} clauses...")

        # Determine which risk categories apply
        risk_cats = []
        for category, keywords in risk_categories.items():
            if any(keyword in clause.lower() for keyword in keywords):
                risk_cats.append(category)

        # Determine which contract categories apply
        contract_cats = identify_clause_types(clause)

        # Only proceed if we have a risk or contract category
        if risk_cats or contract_cats:
            # Use sentiment analysis as a risk detector
            # Truncate long texts to max 512 tokens for the model
            truncated_clause = ' '.join(clause.split()[:500])  # Simple approximation of tokens

            try:
                sentiment = risk_classifier(truncated_clause)[0]
            except RuntimeError:
                # If we still get an error, use an even shorter version
                truncated_clause = ' '.join(clause.split()[:200])
                try:
                    sentiment = risk_classifier(truncated_clause)[0]
                except:
                    # Fallback value if classification fails
                    sentiment = {"label": "NEGATIVE", "score": 0.6}

            # Check for high-risk terms
            is_high_risk, high_risk_term = is_high_risk_term(clause)

            # Assess time sensitivity
            time_frame = is_time_sensitive(clause)

            # Determine risk level with updated logic
            if is_high_risk or (sentiment["label"] == "NEGATIVE" and sentiment["score"] > 0.75):
                risk_level = "High"
            elif sentiment["score"] > 0.6 or any(cat in ["Liability", "Termination", "Intellectual Property"] for cat in risk_cats):
                risk_level = "Medium"
            else:
                # Lower risk for clauses that can be monitored or addressed later
                risk_level = "Low" if time_frame in ["Medium-term", "Long-term"] else "Medium"

            # Adjust risk level based on contract categories
            high_risk_contract_categories = [
                "Uncapped Liability", "Termination for Convenience", "Non-Compete",
                "Exclusivity", "IP Ownership", "Minimum Commitment"
            ]

            if any(cat in high_risk_contract_categories for cat in contract_cats) and risk_level != "High":
                risk_level = "Medium"  # At minimum, these are medium risk

            # Get web search context for the primary category (if available)
            web_context = ""
            primary_category = risk_cats[0] if risk_cats else (contract_cats[0] if contract_cats else "General")

            # Only do web search for higher risk items to avoid rate limiting
            if risk_level in ["High", "Medium"]:
                try:
                    web_context = get_legal_context(clause, primary_category)
                except Exception as e:
                    print(f"Error during web search: {str(e)}")
                    web_context = "Web search unavailable."

            risky_clauses.append({
                "clause": clause,
                "risk_categories": risk_cats,
                "contract_categories": contract_cats,
                "risk_level": risk_level,
                "sentiment_score": sentiment["score"],
                "high_risk_term": high_risk_term if is_high_risk else None,
                "time_sensitivity": time_frame,
                "web_context": web_context
            })

    return risky_clauses

# Generate clause analysis with recommendations
def analyze_clause(clause_info):
    """Analyze a potentially risky clause with warnings about company impact."""

    clause = clause_info["clause"]
    risk_cats = clause_info.get("risk_categories", [])
    contract_cats = clause_info.get("contract_categories", [])
    risk_level = clause_info["risk_level"]
    time_frame = clause_info["time_sensitivity"]

    # Create interpretation based on contract and risk categories
    interpretation = ""

    # Contract category based interpretation
    if contract_cats:
        primary_contract_cat = contract_cats[0]
        contract_interpretations = {
            "Termination for Convenience": "WARNING: This clause allows the client to terminate the contract at their discretion, which could leave your company with unrecoverable costs.",
            "Uncapped Liability": "WARNING: This creates unlimited liability for your company, exposing you to significant financial risk.",
            "Minimum Commitment": "CAUTION: Your company would be committed to minimum purchase/delivery requirements that may become burdensome.",
            "IP Ownership": "WARNING: Your intellectual property rights may be transferred to the client, limiting future use of your own technology.",
            "Non-Compete": "CAUTION: This restricts your company's ability to work with other clients in this sector.",
            "Most Favored Nation": "CAUTION: You would be required to offer this client your best terms, potentially affecting other client relationships.",
            "Change of Control": "NOTE: Changes in your company ownership could trigger termination rights for the client."
        }

        if primary_contract_cat in contract_interpretations:
            interpretation = contract_interpretations[primary_contract_cat]
        else:
            interpretation = f"This clause contains {primary_contract_cat} provisions that should be reviewed."

    # Risk category based interpretation if no contract interpretation was found
    if not interpretation and risk_cats:
        primary_risk_cat = risk_cats[0]
        risk_interpretations = {
            "Termination": "WARNING: This clause allows the client to terminate the contract at their sole discretion without cause, which could leave your company with unrecoverable costs.",
            "Liability": "WARNING: This creates significant liability for your company, exposing you to financial risk.",
            "Financial": "CAUTION: The payment terms could impact your cash flow and operating capital.",
            "Timeline": "CAUTION: There are aggressive deadlines but no accommodations for client-caused delays.",
            "Compliance": "NOTE: Compliance requirements may require additional investment.",
            "Scope Creep": "CAUTION: The client can modify project scope without corresponding budget adjustments.",
            "Intellectual Property": "WARNING: Your rights to use your own technology in future projects may be restricted.",
            "Data Security": "CAUTION: Your company would be liable for data security even with client-provided systems."
        }

        if primary_risk_cat in risk_interpretations:
            interpretation = risk_interpretations[primary_risk_cat]
        else:
            interpretation = "This clause contains potentially problematic language that should be reviewed."

    # If no interpretation was generated yet, use a generic one
    if not interpretation:
        interpretation = "This clause contains terms that should be carefully reviewed."

    # Add high risk term detail if present
    if clause_info.get("high_risk_term"):
        interpretation += f" Specifically, the term '{clause_info['high_risk_term']}' creates significant exposure for your company."

    # Generate recommendation based on risk level and time sensitivity
    recommendations = {
        "High": [
            f"URGENT: Request modification or removal of this clause before proceeding. {time_sensitivity[time_frame]['description']}.",
            f"CRITICAL: Propose alternative language that balances responsibilities between both parties. {time_sensitivity[time_frame]['description']}.",
            f"CRUCIAL: This is a red flag that may warrant declining the RFP if not modified. {time_sensitivity[time_frame]['description']}."
        ],
        "Medium": [
            f"IMPORTANT: Negotiate modifications during the proposal phase. {time_sensitivity[time_frame]['description']}.",
            f"ADDRESS SOON: Add protections for your company before contract signing. {time_sensitivity[time_frame]['description']}.",
            f"ACTION NEEDED: Include specific limitations to protect your interests. {time_sensitivity[time_frame]['description']}."
        ],
        "Low": [
            f"MONITOR: This aspect should be monitored during project execution. {time_sensitivity[time_frame]['description']}.",
            f"NOTE: Document your understanding in your proposal for future reference. {time_sensitivity[time_frame]['description']}.",
            f"CONSIDER: Review during contract negotiation, but presents minimal risk. {time_sensitivity[time_frame]['description']}."
        ]
    }

    import random
    recommendation = random.choice(recommendations[risk_level])

    # Add company impact statement for medium and high risks
    if risk_level in ["Medium", "High"]:
        company_impacts = [
            "Failure to address this could result in financial losses for your company.",
            "This clause creates an imbalance of risk that favors the client at your expense.",
            "Your company may face unexpected costs or liabilities if this is not modified.",
            "This represents a potential threat to your business interests and profitability."
        ]
        recommendation += " " + random.choice(company_impacts)

    return {
        "interpretation": interpretation,
        "risk_level": risk_level,
        "recommendation": recommendation,
        "time_sensitivity": time_frame
    }

# Function to assess overall risk
def assess_overall_risk(risky_clauses):
    if not risky_clauses:
        return {
            "overall_risk": "Low",
            "score": 1,
            "recommendation": "This RFP appears to have minimal contractual risks. Standard proposal response should be sufficient.",
            "company_impact": "Minimal impact expected on your company operations or finances."
        }

    # Calculate risk score
    risk_scores = {"High": 0, "Medium": 0, "Low": 0}
    for clause in risky_clauses:
        risk_scores[clause["risk_level"]] += 1

    # Weight the scores
    weighted_score = (risk_scores["High"] * 3 + risk_scores["Medium"] * 2 + risk_scores["Low"]) / len(risky_clauses)

    # Determine overall risk
    if weighted_score > 2.5 or risk_scores["High"] >= 3:
        overall_risk = "High"
        recommendation = "WARNING: This RFP contains significant risks that could negatively impact your company's financial stability, operational capabilities, or legal standing. Before proceeding, you should carefully evaluate these high-risk elements and consider requesting modifications."
        company_impact = "Substantial financial and operational risks are present that could affect profitability and create long-term liabilities for your company."
    elif weighted_score > 1.5 or risk_scores["High"] >= 1 or risk_scores["Medium"] >= 3:
        overall_risk = "Medium"
        recommendation = "CAUTION: This RFP contains several moderate risks that should be addressed in your proposal. While not immediately disqualifying, these issues could impact project profitability and create unexpected challenges for your company."
        company_impact = "Several elements could impact your project margins and create resource challenges if not properly addressed."
    else:
        overall_risk = "Low"
        recommendation = "This RFP contains minor risks that should be noted but are generally manageable. Most elements can be addressed through standard contract negotiation or monitored during project execution."
        company_impact = "Limited impact expected on your operations or finances, with most risks manageable through normal project controls."

    return {
        "overall_risk": overall_risk,
        "score": weighted_score,
        "recommendation": recommendation,
        "company_impact": company_impact,
        "risk_counts": risk_scores
    }

# Generate category risk assessment
def assess_category_risks(risky_clauses):
    if not risky_clauses:
        return {}, {}

    # Initialize risk category scores
    risk_category_scores = {category: {"count": 0, "high": 0, "medium": 0, "low": 0}
                           for category in risk_categories}

    # Initialize contract category scores
    contract_category_scores = {category: {"count": 0, "high": 0, "medium": 0, "low": 0}
                               for category in contract_categories}

    # Count occurrences by category and risk level
    for clause in risky_clauses:
        # Process risk categories
        for category in clause.get("risk_categories", []):
            risk_category_scores[category]["count"] += 1
            risk_category_scores[category][clause["risk_level"].lower()] += 1

        # Process contract categories
        for category in clause.get("contract_categories", []):
            if category in contract_category_scores:
                contract_category_scores[category]["count"] += 1
                contract_category_scores[category][clause["risk_level"].lower()] += 1

    # Calculate risk score and level for each risk category
    risk_category_risks = {}
    for category, scores in risk_category_scores.items():
        if scores["count"] == 0:
            continue

        # Weight the scores
        weighted_score = (scores["high"] * 3 + scores["medium"] * 2 + scores["low"]) / scores["count"]

        # Determine risk level
        if weighted_score > 2.5 or scores["high"] >= 2:
            risk_level = "High"
        elif weighted_score > 1.5 or scores["high"] >= 1 or scores["medium"] >= 2:
            risk_level = "Medium"
        else:
            risk_level = "Low"

        # Add company impact statements
        if risk_level == "High":
            company_impact = "Major impact on your company's operations, finances, or legal standing."
        elif risk_level == "Medium":
            company_impact = "Moderate impact that could affect project profitability or resource allocation."
        else:
            company_impact = "Minor impact that can likely be managed through normal project controls."

        risk_category_risks[category] = {
            "risk_level": risk_level,
            "score": weighted_score,
            "clause_count": scores["count"],
            "high_count": scores["high"],
            "medium_count": scores["medium"],
            "low_count": scores["low"],
            "company_impact": company_impact
        }

    # Calculate risk score and level for each contract category
    contract_category_risks = {}
    for category, scores in contract_category_scores.items():
        if scores["count"] == 0:
            continue

        # Weight the scores
        weighted_score = (scores["high"] * 3 + scores["medium"] * 2 + scores["low"]) / scores["count"]

        # Determine risk level
        if weighted_score > 2.5 or scores["high"] >= 2:
            risk_level = "High"
        elif weighted_score > 1.5 or scores["high"] >= 1 or scores["medium"] >= 2:
            risk_level = "Medium"
        else:
            risk_level = "Low"

        # Add company impact statements
        if risk_level == "High":
            company_impact = "Major impact on your company's operations, finances, or legal standing."
        elif risk_level == "Medium":
            company_impact = "Moderate impact that could affect project profitability or resource allocation."
        else:
            company_impact = "Minor impact that can likely be managed through normal project controls."

        contract_category_risks[category] = {
            "risk_level": risk_level,
            "score": weighted_score,
            "clause_count": scores["count"],
            "high_count": scores["high"],
            "medium_count": scores["medium"],
            "low_count": scores["low"],
            "company_impact": company_impact
        }

    return risk_category_risks, contract_category_risks

# Generate risk report PDF
def generate_risk_report(rfp_text, risky_clauses, overall_risk, risk_category_risks, contract_category_risks, output_filename="RFP_Risk_Analysis.pdf"):
    doc = SimpleDocTemplate(output_filename, pagesize=letter)
    styles = getSampleStyleSheet()

    # Custom styles
    title_style = styles['Title']
    heading1_style = styles['Heading1']
    heading2_style = styles['Heading2']
    normal_style = styles['Normal']

    # Add custom styles
    styles.add(ParagraphStyle(name='Risk_High', parent=styles['Normal'], textColor=colors.red, fontName='Helvetica-Bold'))
    styles.add(ParagraphStyle(name='Risk_Medium', parent=styles['Normal'], textColor=colors.orange))
    styles.add(ParagraphStyle(name='Risk_Low', parent=styles['Normal'], textColor=colors.green))
    styles.add(ParagraphStyle(name='Warning', parent=styles['Normal'], textColor=colors.red, fontName='Helvetica-Bold'))
    styles.add(ParagraphStyle(name='Impact', parent=styles['Normal'], fontName='Helvetica-Oblique'))
    styles.add(ParagraphStyle(name='Industry', parent=styles['Normal'], textColor=colors.blue))

    # Create document elements
    elements = []

    # Title
    elements.append(Paragraph("RFP Risk Analysis Report", title_style))
    elements.append(Spacer(1, 0.25*inch))

    # Date
    elements.append(Paragraph(f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}", normal_style))
    elements.append(Spacer(1, 0.25*inch))

    # Executive Summary
    elements.append(Paragraph("Executive Summary", heading1_style))

    # Overall risk assessment
    risk_color = risk_severity[overall_risk["overall_risk"]]["color"]
    elements.append(Paragraph(f"Overall Risk Assessment: {overall_risk['overall_risk']}",
                             ParagraphStyle(name='RiskHeading', parent=heading2_style, textColor=risk_color)))
    elements.append(Spacer(1, 0.1*inch))
    elements.append(Paragraph(f"Risk Score: {overall_risk['score']:.2f}/3.0", normal_style))
    elements.append(Spacer(1, 0.1*inch))

    # Company impact (highlighted)
    elements.append(Paragraph("Impact on Your Company:", heading2_style))
    elements.append(Paragraph(overall_risk["company_impact"], ParagraphStyle(name='Impact', parent=normal_style)))
    elements.append(Spacer(1, 0.1*inch))

    # Recommendation with warning styling for high risk
    elements.append(Paragraph("Recommendation:", heading2_style))
    if overall_risk["overall_risk"] == "High":
        elements.append(Paragraph(overall_risk["recommendation"], styles["Warning"]))
    else:
        elements.append(Paragraph(overall_risk["recommendation"], normal_style))
    elements.append(Spacer(1, 0.25*inch))

    # Risk summary table
    elements.append(Paragraph("Risk Summary", heading2_style))

    risk_counts = overall_risk.get("risk_counts", {"High": 0, "Medium": 0, "Low": 0})
    total_clauses = sum(risk_counts.values())

    summary_data = [
        ["Risk Level", "Count", "Percentage", "Required Action"],
        ["High Risk", str(risk_counts["High"]), f"{(risk_counts['High']/total_clauses*100) if total_clauses else 0:.1f}%", "Immediate action required"],
        ["Medium Risk", str(risk_counts["Medium"]), f"{(risk_counts['Medium']/total_clauses*100) if total_clauses else 0:.1f}%", "Address before contract"],
        ["Low Risk", str(risk_counts["Low"]), f"{(risk_counts['Low']/total_clauses*100) if total_clauses else 0:.1f}%", "Monitor during execution"],
        ["Total", str(total_clauses), "100.0%", ""]
    ]

    # Create the summary table
    summary_table = Table(summary_data, colWidths=[1.5*inch, 1*inch, 1*inch, 2*inch])
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (1, 1), (2, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BACKGROUND', (0, 1), (-1, 1), colors.lightcoral),
        ('BACKGROUND', (0, 2), (-1, 2), colors.lightsalmon),
        ('BACKGROUND', (0, 3), (-1, 3), colors.lightgreen),
        ('BACKGROUND', (0, 4), (-1, 4), colors.lightgrey),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))

    elements.append(summary_table)
    elements.append(Spacer(1, 0.25*inch))

    # Time-sensitivity analysis
    elements.append(Paragraph("Action Timeline", heading2_style))
    elements.append(Paragraph("Based on the risk analysis, here is when each issue should be addressed:", normal_style))
    elements.append(Spacer(1, 0.1*inch))

    timeline_data = [
        ["Timeframe", "Description", "Number of Issues"],
        ["Immediate", "Before proposal submission", sum(1 for c in risky_clauses if c.get("time_sensitivity") == "Immediate")],
        ["Short-term", "Before contract signing", sum(1 for c in risky_clauses if c.get("time_sensitivity") == "Short-term")],
        ["Medium-term", "Before project start", sum(1 for c in risky_clauses if c.get("time_sensitivity") == "Medium-term")],
        ["Long-term", "During project execution", sum(1 for c in risky_clauses if c.get("time_sensitivity") == "Long-term")]
    ]

    timeline_table = Table(timeline_data, colWidths=[1.5*inch, 2.5*inch, 1.5*inch])
    timeline_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        ('ALIGN', (2, 1), (2, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BACKGROUND', (0, 1), (-1, 1), colors.lightcoral),
        ('BACKGROUND', (0, 2), (-1, 2), colors.lightsalmon),
        ('BACKGROUND', (0, 3), (-1, 3), colors.lightgreen),
        ('BACKGROUND', (0, 4), (-1, 4), colors.lightblue),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))

    elements.append(timeline_table)
    elements.append(Spacer(1, 0.25*inch))
    elements.append(PageBreak())

    # Risk by Category Section
    elements.append(Paragraph("Risk by Category", heading1_style))

    # Contract Categories Section
    elements.append(Paragraph("Contract Clause Categories", heading2_style))

    if contract_category_risks:
        # Prepare data for the contract category risk table
        category_data = [["Category", "Risk Level", "Issues", "Impact on Your Company"]]

        for category, risk in contract_category_risks.items():
            category_data.append([
                category,
                risk["risk_level"],
                str(risk["clause_count"]),
                risk["company_impact"]
            ])

        # Create the category risk table
        category_table = Table(category_data, colWidths=[1.5*inch, 1*inch, 0.75*inch, 3*inch])

        # Define table styles with colors based on risk level
        table_style = [
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
            ('ALIGN', (1, 1), (2, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ]

        # Add color coding for risk levels
        for i, row in enumerate(category_data[1:], 1):
            risk_level = row[1]
            if risk_level == "High":
                table_style.append(('BACKGROUND', (1, i), (1, i), colors.lightcoral))
            elif risk_level == "Medium":
                table_style.append(('BACKGROUND', (1, i), (1, i), colors.lightsalmon))
            else:
                table_style.append(('BACKGROUND', (1, i), (1, i), colors.lightgreen))

        category_table.setStyle(TableStyle(table_style))
        elements.append(category_table)
    else:
        elements.append(Paragraph("No contract category risks identified.", normal_style))

    elements.append(Spacer(1, 0.25*inch))

    # Risk Categories Section
    elements.append(Paragraph("Risk Categories", heading2_style))

    if risk_category_risks:
        # Prepare data for the risk category table
        category_data = [["Category", "Risk Level", "Issues", "Impact on Your Company"]]

        for category, risk in risk_category_risks.items():
            category_data.append([
                category,
                risk["risk_level"],
                str(risk["clause_count"]),
                risk["company_impact"]
            ])

        # Create the category risk table
        category_table = Table(category_data, colWidths=[1.5*inch, 1*inch, 0.75*inch, 3*inch])

        # Define table styles with colors based on risk level
        table_style = [
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
            ('ALIGN', (1, 1), (2, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ]

        # Add color coding for risk levels
        for i, row in enumerate(category_data[1:], 1):
            risk_level = row[1]
            if risk_level == "High":
                table_style.append(('BACKGROUND', (1, i), (1, i), colors.lightcoral))
            elif risk_level == "Medium":
                table_style.append(('BACKGROUND', (1, i), (1, i), colors.lightsalmon))
            else:
                table_style.append(('BACKGROUND', (1, i), (1, i), colors.lightgreen))

        category_table.setStyle(TableStyle(table_style))
        elements.append(category_table)
    else:
        elements.append(Paragraph("No risk category risks identified.", normal_style))

    elements.append(Spacer(1, 0.25*inch))
    elements.append(PageBreak())

    # Detailed Risk Analysis
    elements.append(Paragraph("Detailed Risk Analysis", heading1_style))

    # First show high-risk clauses
    high_risk_clauses = [c for c in risky_clauses if c['risk_level'] == 'High']
    if high_risk_clauses:
        elements.append(Paragraph("⚠️ HIGH PRIORITY RISKS - IMMEDIATE ACTION REQUIRED", ParagraphStyle(name='HighPriority', parent=heading2_style, textColor=colors.red)))
        elements.append(Paragraph("The following high-risk clauses require immediate attention before proceeding with this RFP:", normal_style))
        elements.append(Spacer(1, 0.1*inch))

        for i, clause_info in enumerate(high_risk_clauses, 1):
            # Get clause analysis
            analysis = analyze_clause(clause_info)

            # Add clause details
            primary_category = (clause_info.get('contract_categories', ['']) + clause_info.get('risk_categories', ['']))[0] or "General"
            elements.append(Paragraph(f"Critical Risk #{i}: {primary_category}", styles["Risk_High"]))
            elements.append(Spacer(1, 0.1*inch))

            # Categories
            contract_cats = ", ".join(clause_info.get("contract_categories", []))
            risk_cats = ", ".join(clause_info.get("risk_categories", []))

            if contract_cats:
                elements.append(Paragraph(f"Contract Categories: {contract_cats}", styles["Italic"]))
            if risk_cats:
                elements.append(Paragraph(f"Risk Categories: {risk_cats}", styles["Italic"]))
            elements.append(Spacer(1, 0.1*inch))

            # Original clause
            elements.append(Paragraph("Original Text:", styles["Heading3"]))
            elements.append(Paragraph(clause_info["clause"], normal_style))
            elements.append(Spacer(1, 0.1*inch))

            # Analysis
            elements.append(Paragraph("Impact on Your Company:", styles["Heading3"]))
            elements.append(Paragraph(analysis["interpretation"], styles["Warning"]))
            elements.append(Spacer(1, 0.1*inch))

            # Add web context if available
            if "web_context" in clause_info and len(clause_info["web_context"]) > 50:
                elements.append(Paragraph("Industry & Legal Context:", styles["Heading3"]))
                # Truncate to first 300 chars if too long
                web_text = clause_info["web_context"]
                if len(web_text) > 300:
                    web_text = web_text[:300] + "..."
                elements.append(Paragraph(web_text, styles["Industry"]))
                elements.append(Spacer(1, 0.1*inch))

            # Recommendation
            elements.append(Paragraph("Recommended Action:", styles["Heading3"]))
            elements.append(Paragraph(analysis["recommendation"], normal_style))

            elements.append(Spacer(1, 0.25*inch))

        elements.append(PageBreak())

    # Medium risk clauses
    medium_risk_clauses = [c for c in risky_clauses if c['risk_level'] == 'Medium']
    if medium_risk_clauses:
        elements.append(Paragraph("⚠️ MEDIUM PRIORITY RISKS - ADDRESS BEFORE CONTRACT", ParagraphStyle(name='MediumPriority', parent=heading2_style, textColor=colors.orange)))
        elements.append(Paragraph("The following clauses present moderate risks that should be addressed in your proposal response:", normal_style))
        elements.append(Spacer(1, 0.1*inch))

        for i, clause_info in enumerate(medium_risk_clauses, 1):
            # Get clause analysis
            analysis = analyze_clause(clause_info)

            # Add clause details
            primary_category = (clause_info.get('contract_categories', ['']) + clause_info.get('risk_categories', ['']))[0] or "General"
            elements.append(Paragraph(f"Moderate Risk #{i}: {primary_category}", styles["Risk_Medium"]))
            elements.append(Spacer(1, 0.1*inch))

            # Categories
            contract_cats = ", ".join(clause_info.get("contract_categories", []))
            risk_cats = ", ".join(clause_info.get("risk_categories", []))

            if contract_cats:
                elements.append(Paragraph(f"Contract Categories: {contract_cats}", styles["Italic"]))
            if risk_cats:
                elements.append(Paragraph(f"Risk Categories: {risk_cats}", styles["Italic"]))
            elements.append(Spacer(1, 0.1*inch))

            # Original clause
            elements.append(Paragraph("Original Text:", styles["Heading3"]))
            elements.append(Paragraph(clause_info["clause"], normal_style))
            elements.append(Spacer(1, 0.1*inch))

            # Analysis
            elements.append(Paragraph("Impact on Your Company:", styles["Heading3"]))
            elements.append(Paragraph(analysis["interpretation"], normal_style))
            elements.append(Spacer(1, 0.1*inch))

            # Recommendation
            elements.append(Paragraph("Recommended Action:", styles["Heading3"]))
            elements.append(Paragraph(analysis["recommendation"], normal_style))

            elements.append(Spacer(1, 0.25*inch))

            # Page break every 3 medium risk clauses for readability
            if i % 3 == 0 and i < len(medium_risk_clauses):
                elements.append(PageBreak())

        # Add page break after medium risks
        if len(medium_risk_clauses) > 0:
            elements.append(PageBreak())

    # Low risk clauses
    low_risk_clauses = [c for c in risky_clauses if c['risk_level'] == 'Low']
    if low_risk_clauses:
        elements.append(Paragraph("✓ LOW PRIORITY RISKS - MONITOR DURING EXECUTION", ParagraphStyle(name='LowPriority', parent=heading2_style, textColor=colors.green)))
        elements.append(Paragraph("The following items present minimal risk but should be monitored during project execution:", normal_style))
        elements.append(Spacer(1, 0.1*inch))

        for i, clause_info in enumerate(low_risk_clauses, 1):
            # Get clause analysis
            analysis = analyze_clause(clause_info)

            # Add clause details - more compact for low risk
            primary_category = (clause_info.get('contract_categories', ['']) + clause_info.get('risk_categories', ['']))[0] or "General"
            elements.append(Paragraph(f"Minor Risk #{i}: {primary_category}", styles["Risk_Low"]))

            # Categories (combined)
            all_cats = clause_info.get("contract_categories", []) + clause_info.get("risk_categories", [])
            if all_cats:
                elements.append(Paragraph(f"Categories: {', '.join(all_cats)}", styles["Italic"]))

            elements.append(Paragraph(f"Original Text: {clause_info['clause']}", normal_style))
            elements.append(Paragraph(f"Recommendation: {analysis['recommendation']}", normal_style))
            elements.append(Spacer(1, 0.15*inch))

            # Add page break every 5 low risk clauses for readability
            if i % 5 == 0 and i < len(low_risk_clauses):
                elements.append(PageBreak())

    # If no risky clauses were found
    if not risky_clauses:
        elements.append(Paragraph("No significant risks identified in the RFP document.", normal_style))

    # Conclusion and next steps
    elements.append(PageBreak())
    elements.append(Paragraph("Conclusion and Next Steps", heading1_style))

    # Generate appropriate conclusion based on overall risk
    if overall_risk["overall_risk"] == "High":
        conclusion = """
        This RFP presents significant risks to your company. Before proceeding with a response, consider:

        1. Requesting clarification or modifications to the most problematic clauses
        2. Preparing alternative language for high-risk terms in your proposal
        3. Evaluating if the potential business opportunity justifies the identified risks
        4. Consulting legal counsel regarding the most concerning contractual elements

        If you decide to proceed, ensure your pricing accounts for the additional risk exposure.
        """
    elif overall_risk["overall_risk"] == "Medium":
        conclusion = """
        This RFP presents moderate risks that should be addressed but are likely manageable. Consider:

        1. Including specific language in your proposal to address the identified risks
        2. Preparing negotiation points for the contract phase
        3. Building appropriate contingencies into your project plan and pricing
        4. Documenting your assumptions regarding ambiguous requirements

        With proper planning and risk management, this opportunity can likely be pursued successfully.
        """
    else:
        conclusion = """
        This RFP presents minimal risks and appears to be a standard business opportunity. Consider:

        1. Noting the identified items for monitoring during project execution
        2. Including standard protections in your proposal and contract
        3. Proceeding with normal proposal development processes

        No special risk mitigation appears necessary for this opportunity.
        """

    elements.append(Paragraph(conclusion, normal_style))

    # Legal disclaimer
    elements.append(Spacer(1, 0.25*inch))
    elements.append(Paragraph("Disclaimer", heading2_style))
    elements.append(Paragraph("This analysis is provided for informational purposes only and does not constitute legal advice. The web search information included in this report is from publicly available sources and may not reflect the most current legal interpretations. Consult with legal counsel before making decisions based on this report.", normal_style))

    # Build and save the document
    doc.build(elements)
    print(f"Report generated: {output_filename}")

    return output_filename

# Main function to run the risk analysis agent
def run_rfp_risk_analysis():
    print("=== RFP Risk Analysis Agent ===")
    print("This agent will analyze your RFP document for potential risks to YOUR COMPANY and generate a detailed report.")

    # Upload RFP document
    rfp_file = upload_rfp_document()

    if not rfp_file:
        return

    # Extract text from PDF
    print("Extracting text from RFP document...")
    rfp_text = extract_text_from_pdf(rfp_file)

    if not rfp_text:
        print("Failed to extract text from the RFP document.")
        return

    # Extract and analyze clauses
    print("Analyzing RFP for potential risks to your company...")
    clauses = extract_clauses(rfp_text)
    print(f"Identified {len(clauses)} potential contractual clauses.")

    # Identify risky clauses
    risky_clauses = identify_risky_clauses(clauses)
    print(f"Detected {len(risky_clauses)} clauses with potential risks to your company.")

    # Assess overall risk
    overall_risk = assess_overall_risk(risky_clauses)
    print(f"Overall risk assessment: {overall_risk['overall_risk']}")

    # Assess category risks
    risk_category_risks, contract_category_risks = assess_category_risks(risky_clauses)

    # Generate risk report
    print("Generating comprehensive risk analysis report...")
    report_file = generate_risk_report(rfp_text, risky_clauses, overall_risk, risk_category_risks, contract_category_risks)

    # Download the report
    files.download(report_file)

    # Print summary
    print("\nRisk Analysis Summary:")
    print(f"Total clauses analyzed: {len(clauses)}")
    print(f"Risky clauses identified: {len(risky_clauses)}")
    print(f"Overall risk level: {overall_risk['overall_risk']}")

    if overall_risk['overall_risk'] == "High":
        print("\n⚠️ WARNING: This RFP contains significant risks to your company!")

    print(f"\nRecommendation: {overall_risk['recommendation'][:100]}...")
    print("\nDownload the full report for detailed analysis and recommendations.")

# Run the analysis agent
run_rfp_risk_analysis()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Token indices sequence length is longer than the specified maximum sequence length for this model (697 > 512). Running this sequence through the model will result in indexing errors


Processed 220 of 227 clauses...
Detected 227 clauses with potential risks to your company.
Overall risk assessment: High
Generating comprehensive risk analysis report...
Report generated: RFP_Risk_Analysis.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Risk Analysis Summary:
Total clauses analyzed: 227
Risky clauses identified: 227
Overall risk level: High

⚠️ WARNING: This RFP contains significant risks to your company!

Recommendation: WARNING: This RFP contains significant risks that could negatively impact your company's financial s...

Download the full report for detailed analysis and recommendations.


In [4]:
{
    "cell_type": "code",
    "metadata": {
        "id": "save_results",
        "language": "python"
    },
    "source": [
        "# Import or create the storage module\n",
        "!pip install -q ipynb-py-convert\n",
        "import os\n",
        "import json\n",
        "import datetime\n",
        "\n",
        "# Define function to save results\n",
        "def save_risk_analysis_results(rfp_name, clauses, risky_clauses, overall_risk, risk_category_risks, contract_category_risks):\n",
        "    \"\"\"Save the risk analysis results to a JSON file\"\"\"\n",
        "    storage_dir = \"risk_analysis_results\"\n",
        "    \n",
        "    # Create storage directory if it doesn't exist\n",
        "    if not os.path.exists(storage_dir):\n",
        "        os.makedirs(storage_dir)\n",
        "    \n",
        "    timestamp = datetime.datetime.now().strftime(\"%Y%m%d_%H%M%S\")\n",
        "    filename = f\"{storage_dir}/{rfp_name}_{timestamp}.json\"\n",
        "    \n",
        "    # Prepare results data\n",
        "    results = {\n",
        "        \"rfp_name\": rfp_name,\n",
        "        \"analysis_date\": datetime.datetime.now().isoformat(),\n",
        "        \"total_clauses\": len(clauses),\n",
        "        \"risky_clauses_count\": len(risky_clauses),\n",
        "        \"overall_risk\": overall_risk,\n",
        "        \"risk_category_risks\": risk_category_risks,\n",
        "        \"contract_category_risks\": contract_category_risks,\n",
        "        \"risky_clauses\": risky_clauses\n",
        "    }\n",
        "    \n",
        "    # Save to JSON file\n",
        "    with open(filename, 'w') as f:\n",
        "        json.dump(results, f, indent=2)\n",
        "    \n",
        "    print(f\"Results saved to {filename}\")\n",
        "    return filename"
      ],
      "outputs": []
},
{
    "cell_type": "code",
    "metadata": {
        "id": "modified_risk_analysis",
        "language": "python"
    },
    "source": [
        "# Modify the main risk analysis function to save results\n",
        "def run_rfp_risk_analysis_with_storage():\n",
        "    print(\"=== RFP Risk Analysis Agent ===\")\n",
        "    print(\"This agent will analyze your RFP document for potential risks to YOUR COMPANY and generate a detailed report.\")\n",
        "    \n",
        "    # Upload RFP document\n",
        "    rfp_file = upload_rfp_document()\n",
        "    \n",
        "    if not rfp_file:\n",
        "        return\n",
        "    \n",
        "    # Use the file name without extension as RFP name\n",
        "    rfp_name = os.path.splitext(rfp_file)[0]\n",
        "    \n",
        "    # Extract text from PDF\n",
        "    print(\"Extracting text from RFP document...\")\n",
        "    rfp_text = extract_text_from_pdf(rfp_file)\n",
        "    \n",
        "    if not rfp_text:\n",
        "        print(\"Failed to extract text from the RFP document.\")\n",
        "        return\n",
        "    \n",
        "    # Extract and analyze clauses\n",
        "    print(\"Analyzing RFP for potential risks to your company...\")\n",
        "    clauses = extract_clauses(rfp_text)\n",
        "    print(f\"Identified {len(clauses)} potential contractual clauses.\")\n",
        "    \n",
        "    # Identify risky clauses\n",
        "    risky_clauses = identify_risky_clauses(clauses)\n",
        "    print(f\"Detected {len(risky_clauses)} clauses with potential risks to your company.\")\n",
        "    \n",
        "    # Assess overall risk\n",
        "    overall_risk = assess_overall_risk(risky_clauses)\n",
        "    print(f\"Overall risk assessment: {overall_risk['overall_risk']}\")\n",
        "    \n",
        "    # Assess category risks\n",
        "    risk_category_risks, contract_category_risks = assess_category_risks(risky_clauses)\n",
        "    \n",
        "    # Generate risk report\n",
        "    print(\"Generating comprehensive risk analysis report...\")\n",
        "    report_file = generate_risk_report(rfp_text, risky_clauses, overall_risk, risk_category_risks, contract_category_risks)\n",
        "    \n",
        "    # Save results to JSON file for other applications\n",
        "    results_file = save_risk_analysis_results(rfp_name, clauses, risky_clauses, overall_risk, \n",
        "                                             risk_category_risks, contract_category_risks)\n",
        "    \n",
        "    # Download the report\n",
        "    files.download(report_file)\n",
        "    \n",
        "    # Print summary\n",
        "    print(\"\\nRisk Analysis Summary:\")\n",
        "    print(f\"Total clauses analyzed: {len(clauses)}\")\n",
        "    print(f\"Risky clauses identified: {len(risky_clauses)}\")\n",
        "    print(f\"Overall risk level: {overall_risk['overall_risk']}\")\n",
        "    \n",
        "    if overall_risk['overall_risk'] == \"High\":\n",
        "        print(\"\\n⚠️ WARNING: This RFP contains significant risks to your company!\")\n",
        "    \n",
        "    print(f\"\\nRecommendation: {overall_risk['recommendation'][:100]}...\")\n",
        "    print(\"\\nDownload the full report for detailed analysis and recommendations.\")\n",
        "    print(f\"\\nResults saved for other applications in: {results_file}\")\n",
        "    \n",
        "    return results_file"
      ],
      "outputs": []
},
{
    "cell_type": "code",
    "metadata": {
        "id": "run_analysis",
        "language": "python"
    },
    "source": [
        "# Run the analysis with storage\n",
        "results_file = run_rfp_risk_analysis_with_storage()"
      ],
      "outputs": []
}

{'cell_type': 'code',
 'metadata': {'id': 'run_analysis', 'language': 'python'},
 'source': ['# Run the analysis with storage\n',
  'results_file = run_rfp_risk_analysis_with_storage()'],
 'outputs': []}